<a href="https://colab.research.google.com/github/sw6820/swm_prototype/blob/main/Oddiya_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------------------------------------------------------
# [셀 1] 설정 및 환경 구성 (수정본)
# ------------------------------------------------------------------------------

# 1. 필수 라이브러리 설치
# ⭐️ 수정된 부분: 의존성 충돌 해결을 위해 'google-ai-generativelanguage' 버전을 명시합니다.
!pip install -q langchain langgraph langchain_google_genai beautifulsoup4 requests google-ai-generativelanguage==0.6.15

# 2. 라이브러리 임포트
import os
import glob
import subprocess
import shutil
import json
import requests
from bs4 import BeautifulSoup
from PIL import Image
from typing import TypedDict, List
import google.generativeai as genai
from google.colab import drive
from google.colab import userdata

# ⭐️ 수정된 부분: LangChain의 경고 메시지에 따라 pydantic v2에서 직접 임포트합니다.
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser

from langgraph.graph import StateGraph, END

# 3. 구글 드라이브 마운트
try:
    drive.mount('/content/drive')
    print("✅ 구글 드라이브 마운트 성공.")
except Exception as e:
    print(f"❗️ 구글 드라이브 마운트 실패: {e}")

# 4. LangSmith 설정
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "AI Video Shorts Pipeline"

# 5. 경로 및 영상 속성 설정
IMAGE_FOLDER = '/content/drive/MyDrive/images'
MUSIC_FOLDER = '/content/drive/MyDrive/music'
OUTPUT_VIDEO = 'output_shorts_langgraph.mp4'
TEMP_DIR = "temp_processing"

IMAGE_DURATION = 3
TRANSITION_DURATION = 1
TARGET_FPS = 30

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 구글 드라이브 마운트 성공.


In [ ]:
# ------------------------------------------------------------------------------
# [셀 2] LangGraph 상태 및 노드(핵심 로직) 정의 (수정본)
# ------------------------------------------------------------------------------

import base64
import io
from pydantic import BaseModel, Field

# --- 1. 그래프의 상태 정의 ---
class GraphState(TypedDict):
    image_paths: List[str]
    music_library: List[str]
    creative_plan: dict
    chosen_music_path: str
    silent_video_path: str

# --- 2. AI 상호작용을 위한 데이터 모델 정의 ---
class CreativePlan(BaseModel):
    title: str = Field(description="A short, catchy title for the video.")
    transitions: List[str] = Field(description="A list of transition effects.")
    music_style_description: str = Field(description="A detailed description of the perfect music style.")

class MusicChoice(BaseModel):
    chosen_filename: str = Field(description="The single best file name from the list.")

# --- 3. 헬퍼 함수 정의 ---
def image_to_base64_uri(pil_image):
    buffered = io.BytesIO()
    pil_image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/jpeg;base64,{img_str}"

def run_ffmpeg(command):
    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("\n❗️ FFmpeg 실행 중 오류가 발생했습니다.")
        print("--- FFmpeg Command ---")
        print(' '.join(f"'{arg}'" if ' ' in arg else arg for arg in command))
        print("\n--- FFmpeg Error ---")
        print(e.stderr)
        raise e

# --- 4. 파이프라인 노드(함수) 정의 ---
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=userdata.get('GOOGLE_API_KEY'))

def load_initial_data(state: GraphState) -> GraphState:
    print("--- 1. 초기 데이터 로드 중... ---")
    image_files = sorted(glob.glob(os.path.join(IMAGE_FOLDER, '*.jpeg')))
    music_files = os.listdir(MUSIC_FOLDER)
    if len(image_files) < 2: raise ValueError("이미지가 2장 미만입니다.")
    if not music_files: raise ValueError("음악 파일이 없습니다.")
    return {"image_paths": image_files, "music_library": music_files}

def get_creative_plan_node(state: GraphState) -> GraphState:
    print("--- 2. AI 영상 계획 수립 중... ---")
    llm_with_tools = llm.with_structured_output(CreativePlan)
    image_messages = [{"type": "image_url", "image_url": image_to_base64_uri(Image.open(p))} for p in state["image_paths"]]
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are a creative director. Create a plan for a short video. Available transitions: {AVAILABLE_TRANSITIONS}"),
        ("human", [{"type": "text", "text": "Analyze these images and create a plan using the 'CreativePlan' tool."}, *image_messages])
    ])
    chain = prompt | llm_with_tools
    try:
        plan = chain.invoke({})
        plan_dict = plan.model_dump()
    except Exception as e:
        print(f"❗️ AI 영상 계획 생성 중 오류 발생, 기본값을 사용합니다: {e}")
        plan_dict = {}
    final_plan = {
        "title": plan_dict.get("title", "My Awesome Slideshow"),
        "transitions": plan_dict.get("transitions", ['fade'] * (len(state["image_paths"]) - 1)),
        "music_style_description": plan_dict.get("music_style_description", "upbeat happy background music")
    }
    return {"creative_plan": final_plan}

def choose_music_node(state: GraphState) -> GraphState:
    print("--- 3. AI 음악 선택 중... ---")
    prompt = ChatPromptTemplate.from_template("""
    You are a Music Director. Select the best song for a video.
    1. Desired music style: "{description}"
    2. Available files: {library}
    Your answer MUST be only the chosen file name from the list. No extra text.
    """)
    chain = prompt | llm
    response = chain.invoke({
        "description": state["creative_plan"]["music_style_description"],
        "library": state["music_library"]
    })
    chosen_filename = response.content.strip()
    if chosen_filename in state["music_library"]:
        chosen_path = os.path.join(MUSIC_FOLDER, chosen_filename)
        print(f"   - AI 선택: {chosen_filename}")
    else:
        chosen_filename_default = state["music_library"][0]
        chosen_path = os.path.join(MUSIC_FOLDER, chosen_filename_default)
        print(f"   - ⚠️ AI의 선택 '{chosen_filename}'을(를) 찾을 수 없어 기본값 '{chosen_filename_default}'(으)로 대체합니다.")
    return {"chosen_music_path": chosen_path}

# ⭐️ 수정된 부분: 실제 FFmpeg 로직을 채워 넣었습니다.
def create_video_node(state: GraphState) -> GraphState:
    """전환 효과가 적용된 무음 영상을 생성하는 노드"""
    print("--- 4. FFmpeg 무음 영상 제작 중... ---")
    image_paths = state["image_paths"]
    transitions = state["creative_plan"]["transitions"]
    base_video_path = os.path.join(TEMP_DIR, "step_0.mp4")

    command = [
        'ffmpeg', '-y', '-loop', '1', '-t', str(IMAGE_DURATION), '-i', image_paths[0],
        '-vf', f"scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2,setsar=1,format=yuv420p,fps={TARGET_FPS}",
        '-c:v', 'libx264', '-r', str(TARGET_FPS), base_video_path
    ]
    run_ffmpeg(command)

    for i in range(1, len(image_paths)):
        print(f"   - {i+1}번째 이미지 합성 중...")
        input_video = base_video_path
        output_video_temp = os.path.join(TEMP_DIR, f"step_{i}.mp4")
        video_duration = (i * IMAGE_DURATION) - ((i - 1) * TRANSITION_DURATION)

        filter_complex_str = (
            f"[0:v]settb=AVTB[v0];"
            f"[1:v]scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2,setsar=1,format=yuv420p,fps={TARGET_FPS},settb=AVTB[v1];"
            f"[v0][v1]xfade=transition={transitions[i-1]}:duration={TRANSITION_DURATION}:offset={video_duration - TRANSITION_DURATION}"
        )
        command = [
            'ffmpeg', '-y', '-i', input_video, '-loop', '1', '-t', str(IMAGE_DURATION), '-i', image_paths[i],
            '-filter_complex', filter_complex_str,
            '-c:v', 'libx264', '-r', str(TARGET_FPS), output_video_temp
        ]
        run_ffmpeg(command)
        base_video_path = output_video_temp

    final_silent_path = os.path.join(TEMP_DIR, "final_silent.mp4")
    shutil.copy(base_video_path, final_silent_path)
    return {"silent_video_path": final_silent_path}

# ⭐️ 수정된 부분: 실제 FFmpeg 로직을 채워 넣었습니다.
def add_music_node(state: GraphState) -> GraphState:
    """최종적으로 음악을 영상에 합성하는 노드"""
    print("--- 5. FFmpeg 음악 합성 중... ---")
    music_path = state["chosen_music_path"]
    silent_video_path = state["silent_video_path"]

    if not music_path or not os.path.exists(music_path):
        print("⚠️ 음악 파일이 없어 음악 없이 영상을 최종 저장합니다.")
        shutil.copy(silent_video_path, OUTPUT_VIDEO)
        return {} # 상태 변경 없이 종료

    command = [
        'ffmpeg', '-y', '-i', silent_video_path, '-i', music_path,
        '-c:v', 'copy',
        '-c:a', 'aac',
        '-shortest',
        OUTPUT_VIDEO
    ]
    run_ffmpeg(command)
    return {}

In [ ]:
# ------------------------------------------------------------------------------
# [셀 3] 그래프 구성 및 실행 (피드백 기능 추가)
# ------------------------------------------------------------------------------
from langsmith import Client

# 1. 그래프 생성
workflow = StateGraph(GraphState)

# 2. 노드들을 그래프에 추가
workflow.add_node("load_data", load_initial_data)
workflow.add_node("get_plan", get_creative_plan_node)
workflow.add_node("choose_music", choose_music_node)
workflow.add_node("create_video", create_video_node)
workflow.add_node("add_music", add_music_node)

# 3. 노드들을 순서대로 연결
workflow.set_entry_point("load_data")
workflow.add_edge("load_data", "get_plan")
workflow.add_edge("get_plan", "choose_music")
workflow.add_edge("choose_music", "create_video")
workflow.add_edge("create_video", "add_music")
workflow.add_edge("add_music", END)

# 4. 그래프 컴파일
app = workflow.compile()

# 5. 메인 파이프라인 실행
if __name__ == "__main__":
    if os.path.exists(TEMP_DIR): shutil.rmtree(TEMP_DIR)
    os.makedirs(TEMP_DIR)

    run_id = None # 피드백을 위해 run_id를 저장할 변수

    try:
        initial_state = {}

        print("🚀 LangGraph 파이프라인 실행 시작...")
        # 스트리밍 로그를 보며 각 단계를 실행
        for event in app.stream(initial_state, stream_mode="values"):
            # 마지막 이벤트에서 run_id를 가져옴
            if "__end__" in event:
                run_id = event["__end__"]["run_id"]

            for key, value in event.items():
                print(f"--- 현재 노드: {key} ---")
                # print(value) # 너무 길면 주석 처리
                print("   ...완료.")

        print("\n🎉 LangGraph 파이프라인 실행 완료! 🎉")
        print(f"💾 최종 저장 파일: {OUTPUT_VIDEO}")
        print(f"🆔 이번 실행의 Run ID: {run_id}")

        # ⭐️⭐️ 수정된 부분: LangSmith 피드백 전송 ⭐️⭐️
        if run_id:
            print("\n" + "="*50)
            print("📝 LangSmith에 피드백을 전송합니다...")
            client = Client()

            # 예시 1: 결과물이 마음에 들었는지 (👍/👎)
            # 1 = Like, 0 = Dislike
            feedback_score = 1 # 마음에 들면 1, 안들면 0으로 설정
            client.create_feedback(
                run_id=run_id,
                key="user_satisfaction", # 피드백 종류를 나타내는 키
                score=feedback_score,
                comment="AI가 추천한 제목과 음악이 영상 분위기와 매우 잘 어울렸습니다."
            )
            print("   - 사용자 만족도(👍) 피드백 전송 완료!")

            # 예시 2: AI가 선택한 음악이 적절했는지 (점수 척도)
            # 1~5점 척도로 평가
            music_rating = 5
            client.create_feedback(
                run_id=run_id,
                key="music_choice_rating",
                score=music_rating,
                comment="음악 라이브러리에서 가장 좋은 곡을 골랐네요."
            )
            print(f"   - 음악 선택 점수({music_rating}/5) 피드백 전송 완료!")

            print("\n🔗 LangSmith 추적 링크에서 전송된 피드백을 확인하세요!")
            print("="*50)

    except Exception as e:
        print(f"\n💥 최종 프로세스 중단: {e}")
    finally:
        print("🧹 임시 파일을 정리합니다.")
        if os.path.exists(TEMP_DIR):
            shutil.rmtree(TEMP_DIR)

🚀 LangGraph 파이프라인 실행 시작...
--- 1. 초기 데이터 로드 중... ---
--- 현재 노드: image_paths ---
   ...완료.
--- 현재 노드: music_library ---
   ...완료.
--- 2. AI 영상 계획 수립 중... ---
--- 현재 노드: image_paths ---
   ...완료.
--- 현재 노드: music_library ---
   ...완료.
--- 현재 노드: creative_plan ---
   ...완료.
--- 3. AI 음악 선택 중... ---
   - AI 선택: summer-vacation-pop-music-353703.mp3
--- 현재 노드: image_paths ---
   ...완료.
--- 현재 노드: music_library ---
   ...완료.
--- 현재 노드: creative_plan ---
   ...완료.
--- 현재 노드: chosen_music_path ---
   ...완료.
--- 4. FFmpeg 무음 영상 제작 중... ---
   - 2번째 이미지 합성 중...
   - 3번째 이미지 합성 중...
   - 4번째 이미지 합성 중...
   - 5번째 이미지 합성 중...
--- 현재 노드: image_paths ---
   ...완료.
--- 현재 노드: music_library ---
   ...완료.
--- 현재 노드: creative_plan ---
   ...완료.
--- 현재 노드: chosen_music_path ---
   ...완료.
--- 현재 노드: silent_video_path ---
   ...완료.
--- 5. FFmpeg 음악 합성 중... ---

🎉 LangGraph 파이프라인 실행 완료! 🎉
💾 최종 저장 파일: output_shorts_langgraph.mp4
🆔 이번 실행의 Run ID: None
🧹 임시 파일을 정리합니다.
